In [11]:
import json
# Load the JSON file
file_path = "monthly_expenses.json"  # Replace with the correct path to your file
with open(file_path, "r") as file:
    data = json.load(file)

monthly_expenses_json_string = json.dumps(data, indent=4) 

In [12]:
from langchain_ollama import ChatOllama
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

#MODEL_NAME = "qwen2.5:7b-instruct"
#model = ChatOllama(model=MODEL_NAME, temperature=0.0)


load_dotenv()
model = ChatOpenAI(model="gpt-4o", temperature=0.7)
#model = ChatOpenAI(model="gpt-4o")
# Create specialized agents

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)

report_agent = create_react_agent(
    model=model,
    tools=[],
    name="report_expert",
    prompt=f"You are a report expert. you can give reports based on this data:{monthly_expenses_json_string }"
)

# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent, report_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
        "For report queries, use report_agent."
    ),
    output_mode="full_history"
)



In [13]:
model 

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000024225D031C0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000024225E488E0>, root_client=<openai.OpenAI object at 0x0000024225DE2560>, root_async_client=<openai.AsyncOpenAI object at 0x0000024224C52C20>, model_name='gpt-4o', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [15]:
# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "give me the summary of the report, what is the average spending each parameter each year"
        }
    ]
})

In [16]:
for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

give me the summary of the report, what is the average spending each parameter each year
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_report_expert (call_PowJ4PnzRnqxaRfU8O2mk8LL)
 Call ID: call_PowJ4PnzRnqxaRfU8O2mk8LL
  Args:
================================= Tool Message =================================
Name: transfer_to_report_expert

Successfully transferred to report_expert
================================== Ai Message ==================================
Name: report_expert

To summarize the report, let's calculate the average spending for each category (Food, Transpo, and Daily) for the years 2023 and 2024.

### 2023:
- **Food**: 
  - Total: 8000 + 7500 + 8500 + 8000 + 7800 + 8200 + 8500 + 8700 + 8100 + 8300 + 8600 + 9000 = 99200
  - Average: 99200 / 12 = 8266.67

- **Transpo**: 
  - Total: 2500 + 2300 + 2600 + 2400 + 25

In [31]:
app = workflow.compile()

# Collect all steps
data = 1
for step in app.stream({
    "messages": [
        {
            "role": "user",
            "content": "give me the summary of the report, what is the average spending each parameter each year"
        }
    ]
}):
    print(step) 
    data = step       # Live output
    

{'supervisor': {'messages': [HumanMessage(content='give me the summary of the report, what is the average spending each parameter each year', additional_kwargs={}, response_metadata={}, id='c43518f0-c0da-4bda-9013-2003eba11a78'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Q8prykSsNt7hoeHu87ue4Cnp', 'function': {'arguments': '{}', 'name': 'transfer_to_report_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 141, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BJylgGfLnMutsCR6cu8Vj7gqc8ABR', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-620905fc-efc3-4525-9eeb-3260e4bf1efd-0', tool_calls=[{'n

In [32]:
for m in data['supervisor']['messages']:
    m.pretty_print()

================================ Human Message =================================

give me the summary of the report, what is the average spending each parameter each year
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_report_expert (call_Q8prykSsNt7hoeHu87ue4Cnp)
 Call ID: call_Q8prykSsNt7hoeHu87ue4Cnp
  Args:
================================= Tool Message =================================
Name: transfer_to_report_expert

Successfully transferred to report_expert
================================== Ai Message ==================================
Name: report_expert

Based on the provided data, here's the summary of average spending for each parameter in the years 2023 and 2024:

### 2023 Average Spending
- **Food**: 
  - Total: 98,200
  - Months: 12
  - Average: 8,183.33
- **Transpo**:
  - Total: 31,300
  - Months: 12
  - Average: 2,608.33
- **Daily**:
  - Total: 19,750
  - Months: 12
  - Average: 1,645.83

### 2